In [1]:
import requests

In [2]:
from bs4 import BeautifulSoup

In [3]:
r = requests.get("https://nyctmc.org/multiview2.php")

In [4]:
bs = BeautifulSoup(r.content)

In [5]:
bs

<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.01 Transitional//EN">

<html lang="en" oncontextmenu="return false" onkeydown="return false" onmousedown="return false" onselectstart="return false">
<head>
<title>NYCDOT - Real Time Traffic Cameras</title><!-- InstanceBegin template="/Templates/new_content.dwt" codeOutsideHTMLIsLocked="false" -->
<meta content="Microsoft Visual Studio .NET 7.1" name="GENERATOR"/>
<meta content="C#" name="CODE_LANGUAGE"/>
<meta content="JavaScript" name="vs_defaultClientScript"/>
<script src="scf/js/jquery.js" type="text/javascript"></script>
<!--Keep the words  NYCDOT -  replace the words to the New Application Name-->
<!-- #BeginEditable "apptitle" -->
<style>.ltgrey { COLOR: #999 }
	OL LI { PADDING-BOTTOM: 6px }
	.titlebar { BORDER-BOTTOM: #a0b3de 1px solid }
	.item { BORDER-BOTTOM: #eee 1px solid }
	.hdr { FONT-WEIGHT: bold; BORDER-BOTTOM: #a0b3de 1px solid }
	TR.ruled { BACKGROUND: #f1f2f7 }
	.header_map_title { PADDING-LEFT: 5px;PADDING-TOP: 5px; FONT-SIZ

In [6]:
from tqdm import tqdm

In [7]:
def get_cam_info(span):
    location = span.get_text()
    borough = span.find_previous("font").get_text().replace("\r", "").replace("\n", "").replace("\t", "").strip()
    multiview_id = span.find_previous("input")["value"]
    cam_multiview_addr = "https://nyctmc.org/multiview2.php?listcam={}".format(multiview_id)
    bs_cam = BeautifulSoup(requests.get(cam_multiview_addr).content)
    img_addr = bs_cam.find_all("img", {"id": "repCamView__ct0_imgLink"})[0]["src"]
    return {
        "location": location,
        "borough": borough,
        "multiview_id": multiview_id,
        "img_addr": img_addr}

In [8]:
all_spans = bs.find_all("span", {"class": "OTopTitle"})

In [9]:
import time

In [10]:
all_cams = []
cam_id = 1
for span in tqdm(all_spans):
    cam_info = get_cam_info(span)
    cam_info["cam_id"] = cam_id
    all_cams.append(cam_info)
    cam_id += 1
    time.sleep(1)

100%|██████████| 654/654 [12:58<00:00,  1.18s/it]


In [11]:
all_cams[:5]

[{'location': '1 Ave @ 110 St',
  'borough': 'Manhattan',
  'multiview_id': '368',
  'img_addr': 'http://207.251.86.238/cctv261.jpg',
  'cam_id': 1},
 {'location': '1 Ave @ 14 St',
  'borough': 'Manhattan',
  'multiview_id': '1189',
  'img_addr': 'http://207.251.86.238/cctv1083.jpg',
  'cam_id': 2},
 {'location': '1 Ave @ 23 St',
  'borough': 'Manhattan',
  'multiview_id': '361',
  'img_addr': 'http://207.251.86.238/cctv253.jpg',
  'cam_id': 3},
 {'location': '1 Ave @ 42 St',
  'borough': 'Manhattan',
  'multiview_id': '550',
  'img_addr': 'http://207.251.86.238/cctv490.jpg',
  'cam_id': 4},
 {'location': '1 Ave @ 62 St',
  'borough': 'Manhattan',
  'multiview_id': '940',
  'img_addr': 'http://207.251.86.238/cctv827.jpg',
  'cam_id': 5}]

In [12]:
import json

In [13]:
with open("./cams.json", "w+") as f:
    json.dump(all_cams, f)